In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.commsec import Trades
import datehandler

# Vars
today = datetime.today()

In [2]:
trades = Trades()
trades_df = trades.all
trades_df

Market    Volume  TradePrice  EffectivePrice  Brokerage
Date       Ticker                                                        
2015-01-07 NAN       ASX    1000.0    1.350000        1.369950       20.0
2015-03-26 WOW       ASX     100.0   28.799999       28.999500       20.0
2016-04-28 RMD       ASX     300.0    7.500000        7.566500       20.0
2016-07-06 QBE       ASX     500.0   10.280000       10.319900       20.0
2017-08-03 NAN       ASX    2000.0    2.310700        2.320675       20.0
...                  ...       ...         ...             ...        ...
2020-01-10 VRS       ASX   -4553.0    0.064000        0.061803       10.0
           WZR       ASX    4448.0    0.225000        0.227248       10.0
2020-01-14 DEM       ASX    4170.0    0.240000        0.242398       10.0
           IFN       ASX   -1767.0    0.680000        0.668710       20.0
2020-01-16 IBXO      ASX  100099.0    0.010000        0.010100       10.0

[66 rows x 5 columns]

# Holdings
- All dates from portfolio inception til now as index
- Store ticker and volume for each ticker on each date
- Store closing price for each ticker on each date

In [3]:
# Construct range of dates over portfolio period
p_inception = trades_df.index[0][0]
p_dates = datehandler.date_list(p_inception, today)

# Create base dataframe
df_base_dates = pd.DataFrame(index=p_dates)
df_base_dates.index.name = 'Date'

# Initiatle the main portfolio df as empty
df_pfolio = pd.DataFrame()

# View
df_base

NameError: name 'df_base' is not defined

In [4]:
# Use list of dicts comprehension to build dataframe using logic
rows_list = []
for row in input_rows:

        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update(blah..) 

        rows_list.append(dict1)

df = pd.DataFrame(rows_list)     

SyntaxError: invalid syntax (<ipython-input-4-4bf134849413>, line 8)

In [5]:
# Build list of tickers seen throughout investment period
tickers = list(sorted(set(trades_df.reset_index().Ticker.to_list())))

# Use yfinance to get close prices
lookup_tickers = [f'{ticker}.AX' for ticker in tickers]  # Only supports ASX stocks
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1])
close = prices['Adj Close'] # Using Adj Close instead of Close to account for stocksplits, dividends
close.columns = tickers

# Step through transactions dataframe, ticker by ticker

for ticker in tickers:
    # Get volumes
    vol_df = trades.by_ticker(ticker)['Volume']

    # Merge with base_dates_df and cumulative sum the volumes
    ticker_df = pd.merge_ordered(vol_df, df_base_dates, on='Date').set_index('Date')
    ticker_df = ticker_df.cumsum().ffill()
    ticker_df = ticker_df[ticker_df['Volume'].isna() == False]

    # Merge with close prices
    ticker_df = pd.merge_ordered(ticker_df, close[ticker], on='Date').set_index('Date')
    ticker_df.rename({ticker: 'Close'}, axis=1, inplace=True)
    
    # Merge ticker_df with portfolio_df
    ticker_df['Ticker'] = ticker # create ticker column for later indexing
    df_pfolio = pd.concat([df_pfolio, ticker_df])

# Finalise portfolio
df_pfolio = df_pfolio.reset_index().set_index(['Date','Ticker']).sort_index()

[*********************100%***********************]  35 of 35 completed

2 Failed downloads:
- IBXO.AX: No data found for this date range, symbol may be delisted
- SRS.AX: No data found, symbol may be delisted


In [10]:
df_pfolio.loc['2020-03-26']

Volume      Close
Date       Ticker                     
2020-03-26 A2M          0.0  16.000000
           ACW      20019.0   0.017000
           AMT          0.0   0.155000
           AVH      10225.0   0.480000
           AVZ      42459.0   0.051000
           BRN      13670.0   0.042000
           DEM       8019.0   0.145000
           EHE          0.0   1.280000
           FTT      18199.0   0.001500
           IBXO    100099.0        NaN
           IFN        -27.0   0.400000
           IMM      24414.0   0.115000
           IMU      63116.0   0.019000
           JMS       8220.0   0.220000
           MDR      18886.0   0.405000
           MSB       2017.0   1.290000
           NAN       1027.0   5.210000
           NEA        298.0   0.995000
           OCC       2669.0   0.240000
           OPT        361.0   1.760000
           OZL          0.0   7.160000
           PME         40.0  17.780001
           PNV       3059.0   1.550000
           PTX      31558.0   0.027000
           QBE        -37.0   8.620000
           RAP      14372.0   0.160000
           RHT       9263.0   0.105000
           RMD        213.0  22.820000
           RWC        555.0   2.280000
           SRS          0.0        NaN
           TTT       2067.0   0.545000
           VRS       -105.0   0.022000
           WOW         48.0  36.730000
           WZR       4448.0   0.090000
           ZNO       5004.0   1.480000

In [7]:
ticker = 'NAN'

# Get volumes
vol_df = trades.by_ticker(ticker)['Volume']

# Merge with base_dates_df and cumulative sum the volumes
ticker_df = pd.merge_ordered(vol_df, df_base_dates, on='Date').set_index('Date')
ticker_df = ticker_df.cumsum().ffill()
ticker_df = ticker_df[ticker_df['Volume'].isna() == False]

# Merge with close prices
ticker_df = pd.merge_ordered(ticker_df, close[ticker], on='Date').set_index('Date')
ticker_df.rename({ticker: 'Close'}, axis=1, inplace=True)

# Merge ticker_df with portfolio_df
ticker_df['Ticker'] = ticker # create ticker column for later indexing

ticker_df

,Volume,Close,Ticker
Date,,,
2015-01-07,1000.0,1.350,NAN
2015-01-08,1000.0,1.325,NAN
2015-01-09,1000.0,1.270,NAN
2015-01-12,1000.0,1.225,NAN
2015-01-13,1000.0,1.235,NAN
...,...,...,...
2020-03-23,1027.0,4.370,NAN
2020-03-24,1027.0,4.670,NAN
2020-03-25,1027.0,4.910,NAN


In [8]:
# Use yfinance to get close prices
lookup_tickers = [f'{ticker}.AX' for ticker in tickers]  # Only supports ASX stocks
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1])

[*********************100%***********************]  35 of 35 completed

2 Failed downloads:
- IBXO.AX: No data found for this date range, symbol may be delisted
- SRS.AX: No data found, symbol may be delisted


In [9]:
close = prices['Adj Close']
close.columns = tickers
close

,A2M,ACW,AMT,AVH,AVZ,BRN,DEM,EHE,FTT,IBXO,...,RAP,RHT,RMD,RWC,SRS,TTT,VRS,WOW,WZR,ZNO
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-07,NaN,0.0740,0.055536,0.069718,0.005,0.011118,NaN,3.533724,0.287760,NaN,...,0.010666,0.0400,6.759596,NaN,NaN,NaN,0.159193,24.641617,0.450,1.00
2015-01-08,NaN,0.0610,0.052759,0.067953,0.005,0.011118,NaN,3.622253,0.292477,NaN,...,0.010666,0.0430,6.798389,NaN,NaN,NaN,0.159193,24.386559,0.450,1.00
2015-01-09,NaN,0.0690,0.052759,0.067953,0.005,0.011118,NaN,3.622253,0.301912,NaN,...,0.010666,0.0430,6.934163,NaN,NaN,NaN,0.159193,24.863760,0.450,1.00
2015-01-12,NaN,0.0740,0.052759,0.070601,0.006,0.011118,NaN,3.607497,0.301912,NaN,...,0.010666,0.0420,6.837182,NaN,NaN,NaN,0.154511,24.740349,0.450,1.00
2015-01-13,NaN,0.0690,0.055536,0.070601,0.007,0.011118,NaN,3.607497,0.316064,NaN,...,0.018666,0.0400,6.982653,NaN,NaN,NaN,0.154511,24.600477,0.450,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-20,16.170000,0.0175,0.150000,0.380000,0.048,0.033000,0.120,1.095000,0.002000,NaN,...,0.074000,0.1100,22.240000,1.98,NaN,0.625,0.030000,37.470001,0.089,1.42
2020-03-23,15.400000,0.0160,0.150000,0.360000,0.045,0.035000,0.105,0.905000,0.001500,NaN,...,0.069000,0.0920,20.040001,1.72,NaN,0.550,0.020000,36.450001,0.070,1.28
2020-03-24,16.059999,0.0165,0.150000,0.360000,0.048,0.036000,0.120,1.070000,0.001500,NaN,...,0.145000,0.0950,22.010000,1.74,NaN,0.505,0.021000,36.180000,0.082,1.27
